# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url	category	state
2024-06-21	基于混合现实的物品抓取方法及混合现实设备(An object grasping method and mixed reality equipment based on mixed reality)	China		 Chenglong Fu, Chengjie Zhang, Yuquan Leng, **Guojing Huang**, Yaoyu Cheng, CN116243795B, China, An object grasping method and mixed reality equipment based on mixed reality. 	2024-patent	https://patents.google.com/patent/CN116243795B/zh?oq=CN+116243795+B		patents 	Granted
2024-06-23	Evolutionary Morphology Towards Overconstrained Locomotion via Large-Scale, Multi-Terrain Deep Reinforcement Learning	IEEE/IFToMM International Conference on Reconfigurable Mechanisms and Robots (ReMAR2024), Chicago, USA.		Yenan Chen, Chuye Zhang, Pengxi Gu, Jianuo Qiu, Jiayi Yin, Nuofan Qiu, **Guojing Huang**, Bangchao Huang, Zishang Zhang, Hui Deng, Wei Zhang, Fang Wan, Chaoyang Song. (2024). "Evolutionary Morphology Towards Overconstrained Locomotion via Large-Scale, Multi-Terrain Deep Reinforcement Learnin

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url,category,state
0,2024-06-21,基于混合现实的物品抓取方法及混合现实设备(An object grasping method...,China,NaN,"Chenglong Fu, Chengjie Zhang, Yuquan Leng, **...",2024-patent,https://patents.google.com/patent/CN116243795B...,NaN,patents,Granted
1,2024-06-23,Evolutionary Morphology Towards Overconstraine...,IEEE/IFToMM International Conference on Reconf...,NaN,"Yenan Chen, Chuye Zhang, Pengxi Gu, Jianuo Qiu...",2024-ReMAR,https://ieeexplore.ieee.org/abstract/document/...,NaN,conferences,Published
2,2024-07-08,IMU-Based Prediction of Multiple Grasping Gest...,IEEE International Conference on Advanced Robo...,NaN,"Guotao Gou, Kong Hoi Cheng, Jinxin Sun, Chengy...",2024-ICARM-1,https://ieeexplore.ieee.org/abstract/document/...,,conferences,Published
3,2024-07-08,Flexible Multi-Channel Electrical Stimulation ...,IEEE International Conference on Advanced Robo...,NaN,"Jinxin Sun, **Guojing Huang**, Chengyu Lin, We...",2024-ICARM-2,https://ieeexplore.ieee.org/abstract/document/...,,conferences,Published
4,2024-10-01,Overconstrained Locomotion,International Symposium of Robotics Research (...,NaN,"Haoran Sun, Shihao Feng, Bangchao Huang, Zish...",2024-ISRR,https://dx.doi.org/10.48550/arXiv.2310.09824,,conferences,Published


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""

    md += """\ncategory: """ + item.category
    
    md += """\npermalink: /publication/""" + item.category + """/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"

    md += "\nstate: " + item.state
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    # md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    # if len(str(item.excerpt)) > 5:
    #     md += "\n" + html_escape(item.excerpt) + "\n"

    # if len(str(item.slides_url)) > 5:
    #     md += "\n[Download slides here](" + item.slides_url + ")\n" 

    # if len(str(item.paper_url)) > 5:
    #     md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
    
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2024-06-21-2024-patent.md
2024-06-23-2024-ReMAR.md
2024-07-08-2024-ICARM-1.md
2024-07-08-2024-ICARM-2.md
2024-10-01-2024-ISRR.md
2025-01-17-2025-RSS.md
journal


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
